# Matrix Representations of Automorphism Gates

For any given error correction code, there is no inherently preferable choice of basis for the logical qubits. In this note we impose some external criteria for basis selection in bivariate bicycle codes with a logic unit, and use these to fix a basis. Then we write the automorphism gates in this basis.

## Criteria for logical bases

 - For CSS codes, logical $\bar X$ operators should consist just of physical $X$ operators, and similarly for $Z$.
 - Bivariate bicycle codes possess a ZX-duality symmetry: If you take a physical support of a logical $\bar X$ operator, transpose the torus coordinates and swap the L and R data blocks, then you get the coordinates of a logical $\bar Z$ operator (ad similarly for $Z$ operators). Hence every $\bar X$, $\bar Z$ that define a qubit have a dual $\bar Z', \bar X'$ pair that define a dual qubit. We want the primal qubits to be 0...5, and the dual qubits to be 6...11. (We've also called these unprimed and primed qubits.)
  - A logic unit is attached to a pair of $\bar X, \bar Z$ and their duals $\bar Z', \bar X'$. We'd like these qubits to be qubits 0 and 6.

These criteria do not suffice to define a basis. Here we find *any* basis that matches these criteria. However, all other such bases can be obtained by acting with a logical CNOT circuit on qubits 1...5 and with the same circuit on 7...11.

## Automorphisms

Since automorphisms take $X$ to $X$ and $Z$ to $Z$, they are logical CNOT circuits. CNOT circuits $U$ are specified by a matrix $A \in GF(2)^{12 \times 12}$ via $U\ket{\vec x} = \ket{A \vec x}$. They act on Pauli matrices as $U X(\vec v) U^\dagger = X(A \vec v)$ and  $U Z(\vec v) U^\dagger = Z((A^{-1})^T \vec v)$. Automorphisms do not entangle the primal and dual qubits, and act the same way on them, so $A$ takes the form $\begin{bmatrix} M & 0 \\ 0 & M\end{bmatrix}$ for some $M \in GF(2)^{6 \times 6}$. There are 12 native autmorphism gates, but they generate a group $\cong \mathbb{Z}_6 \otimes \mathbb{Z}_6$ and is hence generated by two elements $\langle x, y \rangle$. Hence we just need two matrices $M_x, M_y \in GF(2)^{6 \times 6}$.

Automorphisms and logic unit measurements are the backbone of the logical gate set. Logic unit measurements conjugated by automorphisms togethe form the ``native measurements''. Since our basis has the logic unit attached to qubits 0 and 6, the matrices $M_x, M_y$ suffice to specify all native measurements. Given these, there is no need to write down the physical renditions of the logical qubits explicitly.

## Physical coordinates of a logical qubit

Below a logical qubit is specified by four polynomials: `Lx`, `Rx` specify $\bar X$, and `Lz`, `Rz` specify $\bar Z$. The gross code's physical qubits have an L and R block, and each physical qubit within each block is indexed by a monomial. Hence a polynomial (over GF(2)) specifies a subset of qubits within a block. $\bar X$ consists of physical $X$ operators supported on `Lx` within L, and on `Rx` within R - we write $\bar X$ = `X(Lx, Rz)`. Similarly $\bar Z$ = `Z(Lz, Rz)`.

In [ ]:
import numpy as np
import warnings
warnings.filterwarnings("error")
from polynomial import Polynomial
from matrix_utils import row_echelon, decompose_row_vector

In [ ]:
toric = True
# ^^ use labeling in appendix of Tour de Gross

if False:
    current_code = "gross"
    order = (12,6)
    x = Polynomial([(1, 0)],order=order)
    y = Polynomial([(0, 1)],order=order)
    if not toric:
        A, B = x**3 + y + y**2, y**3 + x + x**2
    else:
        A, B = 1 + y + x**3*y**-1, 1 + x + x**-1*y**-3
else:
    current_code = "two-gross"
    order = (12,12)
    x = Polynomial([(1, 0)],order=order)
    y = Polynomial([(0, 1)],order=order)
    if not toric:
        A, B = x**3 + y**2 + y**7, y**3 + x + x**2
    else:
        A, B = 1 + y + x**3*y**-1, 1 + x + x**-1*y**-3

In [11]:
zero = Polynomial([],order=order)
one = Polynomial([(0,0)],order=order)

monomials = lambda: Polynomial.iter_monomials(order=order)
cast = lambda x: Polynomial(x,order=order)

def anticommute(Lx:Polynomial, Rx:Polynomial, Lz:Polynomial, Rz:Polynomial) -> bool:
    """Do X(Lz,Rx) and Z(Lz,Rz) anticommute?"""
    return 1 in Lx * Lz.T + Rx * Rz.T


## Obtain a logical basis with a given pivot qubit as the first qubit

In [12]:
def check_logical_qubit(Lx, Rx, Lz, Rz) -> None:
    """Verify that X(Lz,Rx) and Z(Lz,Rz) commute with the stabilizer and anticommute with each other, hence forming a valid logical qubit."""
    assert anticommute(Lx,Rx,Lz,Rz) # provided operators should anticommute
    assert all([not anticommute(m*A, m*B, Lz, Rz) for m in monomials()]) # commutes with X stabilizer
    assert all([not anticommute(Lx, Rx, m*B.T, m*A.T) for m in monomials()]) # commutes with Z stabilizer

# physical support of a logical operator on the L and R blocks
LogicalOperatorSupport = tuple[Polynomial, Polynomial]

def get_block_basis_with_pivot(Lx:Polynomial, Rx:Polynomial, Lz:Polynomial, Rz:Polynomial) -> tuple[list[LogicalOperatorSupport],list[LogicalOperatorSupport]]:
    """Finds a set of logical operators for the primal block, as a list of X supports and a list of Z supports.
    This basis should be symplectic, and feature the pivot qubit as the first qubit.
    To obtain the basis for the dual block, apply the ZX-duality to each qubit. That is, if the output of this function is LX, LZ,
    then the first dual qubit's X operator is X(LZ[0][1].T, LZ[0][0].T), and the Z operator is Z(LX[0][1].T, LX[0][0].T).
    """

    check_logical_qubit(Lx,Rx,Lz,Rz)

    # Get the X checks in matrix form - used to determine if an operator is independent of the stabilizer
    Xops = np.concatenate([A.mat(), B.mat()],axis=1)
    Xops,_ = row_echelon(Xops)
    rank = sum([1 if np.sum(Xops[i,:]%2) > 0 else 0 for i in range(Xops.shape[0])])

    z0 = np.concatenate([(Lz).vec(),(Rz).vec()],axis=1)

    # Obtain candidates for X operators by shifting Lx and Rx around the torus.
    # This method is guaranteed to find operators in the primal block only.
    LX = []
    for m in monomials():
        v = np.concatenate([(Lx*m).vec(),(Rx*m).vec()],axis=1)
        if len(LX) > 0 and np.dot(v,z0.T)%2 == 1:
            # The first operator should anticommute with z0, but the others should not.
            continue
        Xops,_ = row_echelon(np.concatenate([Xops,v]))
        if sum([1 if np.sum(Xops[i,:]%2) > 0 else 0 for i in range(Xops.shape[0])]) > rank:
            # See if adding the operator to our Xops actually increased the rank.
            # If so, it is a new logical operator - add it to the list.
            rank += 1
            LX.append(v)

    dim = order[0]*order[1]

    # Make a table of anticommutation relations between LX and operators of the form Z(m*Lz, m*Rz).
    Z_anticommutations = np.zeros((dim, len(LX)),dtype=int)
    for i,m in enumerate(monomials()):
        for j,v in enumerate(LX):
            w = np.concatenate([(Lz*m).vec(),(Rz*m).vec()],axis=1)
            Z_anticommutations[i,j] = np.dot(v,w.T)[0,0]%2

    # Find linear combinations of the Z operators that form a symplectic basis with LX.
    XA, X = row_echelon(Z_anticommutations, reduced=True)
    # Linear combinations of the m's in Z(m*Lz, m*Rz) are now encoded into X.

    # Extract the coordinates of Lz, Rz from X.
    LZ = []
    for i in range(len(LX)):
        v = np.array([i==j for j in range(len(LX))],dtype=int)
        leftovers, decomp = decompose_row_vector(v,XA,X=X)
        assert np.sum(leftovers%2) == 0
        newz = sum([
            np.concatenate([(Lz*m).vec(),(Rz*m).vec()],axis=1)
            for j,m in enumerate(monomials())
            if decomp[0,j]%2 == 1
        ])
        LZ.append(newz%2)

    # Verify that the result is symplectic
    for i,v in enumerate(LX):
        for j,w in enumerate(LZ):
            assert (np.dot(v,w.T)%2 == 1) == (i == j)

    # Convert the coordinates from vector form to polynomial form.
    LX = [(cast(v[0,:dim]),cast(v[0,dim:])) for v in LX]
    LZ = [(cast(v[0,:dim]),cast(v[0,dim:])) for v in LZ]

    return LX, LZ



In [13]:
def get_block_basis_with_shifts(Lx:Polynomial, Rx:Polynomial, Lz:Polynomial, Rz:Polynomial, xshifts:list[Polynomial], zshifts:list[Polynomial]) -> tuple[list[LogicalOperatorSupport],list[LogicalOperatorSupport]]:
    """Finds a set of logical operators for the primal block, as a list of X supports and a list of Z supports.
    This basis should be symplectic, and feature the pivot qubit as the first qubit.
    To obtain the basis for the dual block, apply the ZX-duality to each qubit. That is, if the output of this function is LX, LZ,
    then the first dual qubit's X operator is X(LZ[0][1].T, LZ[0][0].T), and the Z operator is Z(LX[0][1].T, LX[0][0].T).
    """

    assert len(xshifts) == len(zshifts)
    check_logical_qubit(xshifts[0]*Lx,xshifts[0]*Rx,zshifts[0]*Lz,zshifts[0]*Rz)

    LX = []
    LZ = []

    for alpha in xshifts:
        LX.append((alpha*Lx, alpha*Rx))

    for beta in zshifts:
        LZ.append((beta*Lz, beta*Rz))

    for i,alpha in enumerate(xshifts):
        for j,beta in enumerate(zshifts):
            assert anticommute(alpha*Lx, alpha*Rx, beta*Lz, beta*Rz) == (i == j)

    return LX, LZ


## Extract a linear map for an automorphism

In [14]:
def get_linear_map_for_automorphism(m:Polynomial,Xs: list[LogicalOperatorSupport],Zs: list[LogicalOperatorSupport]) -> np.ndarray:
    """Get the 6x6 matrix M defining an automorphism. Automorphisms are defined by a monomial m, encoding a shift of the torus."""
    out = np.zeros((len(Zs),len(Zs)),dtype=int)

    for i,(lx,rx) in enumerate(Xs):
        for j,(lz,rz) in enumerate(Zs):
            out[j,i] = anticommute(m*lx, m*rx, lz, rz)

    return out

In [15]:
def get_automorphisms_for_2407_18393():
    """Logic unit presented in https://arxiv.org/abs/2407.18393."""
    assert current_code == "gross"

    Lx = y + x*y**5 + x**2*y**2 + x**2*y**4 + x**3*y + x**3*y**2 + x**4*y + x**4*y**2 + x**9*y**3 + x**10 + x**11 + x**11*y**3
    Rx = y**2 + x*y + x*y**5 + x**6
    Lz = x**2*y**3 + x**2*y**5 + x**3 + x**3*y**3 + x**3*y**4 + x**3*y**5
    Rz = x*y**3 + x*y**5 + x**2 + x**2*y**4 + x**3*y**2 + x**3*y**4

    Xs, Zs = get_block_basis_with_pivot(Lx, Rx, Lz, Rz)

    print("x")
    print(get_linear_map_for_automorphism(x,Xs,Zs))

    print("\ny")
    print(get_linear_map_for_automorphism(y,Xs,Zs))

    print("\nBasis:")
    for i,((lx,rx),(lz,rz)) in enumerate(zip(Xs,Zs)):
        print(f"\\bar X_{i} = X({lx},     {rx})")
        print(f"\\bar Z_{i} = Z({lz},     {rz})")

get_automorphisms_for_2407_18393()

x
[[0 0 1 1 0 0]
 [0 0 1 0 1 0]
 [0 0 1 0 0 1]
 [1 1 1 0 0 0]
 [0 0 1 1 1 0]
 [0 1 0 1 0 1]]

y
[[1 0 1 0 0 1]
 [0 0 0 0 0 1]
 [1 1 0 0 0 1]
 [1 0 0 0 0 1]
 [1 0 0 1 0 0]
 [1 0 0 0 1 0]]

Basis:
\bar X_0 = X(y + x*y**5 + x**2*y**2 + x**2*y**4 + x**3*y + x**3*y**2 + x**4*y + x**4*y**2 + x**9*y**3 + x**10 + x**11 + x**11*y**3,     y**2 + x*y + x*y**5 + x**6)
\bar Z_0 = Z(x**2*y**3 + x**2*y**5 + x**3 + x**3*y**3 + x**3*y**4 + x**3*y**5,     x*y**3 + x*y**5 + x**2 + x**2*y**4 + x**3*y**2 + x**3*y**4)
\bar X_1 = X(y**5 + x*y**3 + x**2 + x**2*y**2 + x**3 + x**3*y**5 + x**4 + x**4*y**5 + x**9*y + x**10*y**4 + x**11*y + x**11*y**4,     1 + x*y**3 + x*y**5 + x**6*y**4)
\bar Z_1 = Z(x**2 + x**2*y**3 + x**2*y**4 + x**2*y**5 + x**3 + x**3*y + x**3*y**4 + x**3*y**5 + x**4*y + x**4*y**3,     x + x*y**3 + x*y**4 + x*y**5 + x**2*y + x**2*y**3 + x**3 + x**3*y + x**3*y**2 + x**3*y**3 + x**4*y**2 + x**4*y**3 + x**4*y**4 + x**4*y**5)
\bar X_2 = X(1 + x*y**4 + x**2*y + x**2*y**3 + x**3 + x**3*y + x**4 + x*

In [16]:
def get_automorphisms_for_system_2():
    """Logic unit designed by Ted Yoder in December 2024."""
    assert current_code == "gross"

    Lx = 1 + x + x**2 + x**3 + x**6 + x**7 + x**8 + x**9 + (x + x**5 + x**7 + x**11) * y**3
    Rx = zero
    p = x**5 * y + x**4 * y**2 + x**3 * y**4 + (x**3 + x**4 + x**5 + x**6 + x**7) * y**5
    q = x**3 + x**4 * y + x**4 * y**2 + x**6 * y**5
    Lz = q.T
    Rz = p.T

    Xs, Zs = get_block_basis_with_pivot(Lx, Rx, Lz, Rz)

    print("x")
    print(get_linear_map_for_automorphism(x,Xs,Zs))

    print("\ny")
    print(get_linear_map_for_automorphism(y,Xs,Zs))

    print("\nBasis:")
    for i,((lx,rx),(lz,rz)) in enumerate(zip(Xs,Zs)):
        print(f"\\bar X_{i} = X({lx},     {rx})")
        print(f"\\bar Z_{i} = Z({lz},     {rz})")

get_automorphisms_for_system_2()

x
[[1 1 0 1 1 1]
 [0 0 0 0 1 0]
 [0 0 0 0 1 1]
 [1 1 1 1 0 1]
 [0 1 0 0 1 0]
 [1 1 0 0 1 0]]

y
[[1 0 1 0 1 0]
 [1 0 1 0 0 1]
 [1 1 0 0 1 0]
 [0 0 0 0 1 0]
 [0 0 0 1 1 0]
 [1 0 0 0 0 1]]

Basis:
\bar X_0 = X(1 + x + x*y**3 + x**2 + x**3 + x**5*y**3 + x**6 + x**7 + x**7*y**3 + x**8 + x**9 + x**11*y**3,     0)
\bar Z_0 = Z(x**6*y + x**8*y**4 + x**8*y**5 + x**9,     x**5*y + x**6*y + x**7*y + x**7*y**5 + x**8*y + x**8*y**4 + x**9*y + x**9*y**2)
\bar X_1 = X(y**2 + x*y**2 + x*y**5 + x**2*y**2 + x**3*y**2 + x**5*y**5 + x**6*y**2 + x**7*y**2 + x**7*y**5 + x**8*y**2 + x**9*y**2 + x**11*y**5,     0)
\bar Z_1 = Z(x**6 + x**6*y + x**6*y**2 + x**6*y**4 + x**7*y**2 + x**8 + x**8*y + x**8*y**2 + x**8*y**3 + x**9*y + x**9*y**3 + x**10*y,     x**5 + x**5*y + x**5*y**2 + x**5*y**4 + x**6 + x**6*y + x**6*y**4 + x**7*y + x**7*y**2 + x**7*y**5 + x**8*y**3 + x**8*y**5 + x**9 + x**9*y**2 + x**9*y**3 + x**9*y**4 + x**10*y**2 + x**10*y**3)
\bar X_2 = X(y**3 + x + x*y**3 + x**2*y**3 + x**3*y**3 + x**5 + x**6*

In [33]:
def get_automorphisms_for_system_3():
    """Logic unit designed by Ted Yoder in January 2024."""
    assert current_code == "gross"

    if not toric:
        Lx = x**3 + x**4 + x**5 * y + x**3 * y**2 + x**4 * y**4 + x**5 * y**5
        Rx = x**3 + (x**3 + x**4) * y + x**3 * y**4 + (x**3 + x**4) * y**5
        Lz = ((x**4 + x**8) * y**3 + (1 + x**8) * y**4 + (x + x**9) * y**5).T
        Rz = ((x**7 + x**11) + (1 + x**8) * y + (x**2 + x**10) * y**4).T
    else:
        p = x**4+x**5+x**6*y+x**4*y**2+x**5*y**4+x**6*y**5
        q = x**3+x**4+x**3*y+x**3*y**2+x**4*y**2+x**3*y**5
        r = 1+x**8+x*y+x**9*y+x**3*y**4+x**11*y**4
        s = x+x**9+x**4*y**4+x**8*y**4+y**5+x**8*y**5

        nu = x*y
        Lx = p
        Rx = q
        Lz = nu*s.T
        Rz = nu*r.T

    xshifts = [x**0, x**3*y**5, x**11*y**5, x**10*y, x**5*y**4, x**4*y**2]
    zshifts = [x**0, x**2*y**4, x*y**2, x**2*y**5, x*y, x**3*y]

    # Xs, Zs = get_block_basis_with_pivot(Lx, Rx, Lz, Rz)
    Xs, Zs = get_block_basis_with_shifts(Lx, Rx, Lz, Rz, xshifts, zshifts)

    print("x")
    print(get_linear_map_for_automorphism(x,Xs,Zs))

    print("\ny")
    print(get_linear_map_for_automorphism(y,Xs,Zs))

    print("\nBasis:")
    for i,((lx,rx),(lz,rz)) in enumerate(zip(Xs,Zs)):
        print(f"\\bar X_{i} = X({lx},     {rx})")
        print(f"\\bar Z_{i} = Z({lz},     {rz})")

get_automorphisms_for_system_3()

x
[[0 1 0 1 0 0]
 [0 1 0 0 0 1]
 [0 0 1 1 0 0]
 [1 1 0 1 1 0]
 [0 1 0 0 1 0]
 [1 1 1 1 0 1]]

y
[[1 0 0 0 0 1]
 [1 1 1 0 0 1]
 [0 0 0 0 1 0]
 [0 1 0 0 0 0]
 [0 1 1 0 0 1]
 [0 0 1 1 0 1]]

Basis:
\bar X_0 = X(x**4 + x**5 + x**6*y + x**4*y**2 + x**5*y**4 + x**6*y**5,     x**3 + x**4 + x**3*y + x**3*y**2 + x**4*y**2 + x**3*y**5)
\bar Z_0 = Z(y + x**4*y + x**9*y**3 + x**5*y**3 + x*y**2 + x**5*y**2,     x**5*y + x*y + 1 + x**4 + x**10*y**3 + x**2*y**3)
\bar X_1 = X(x**7*y**5 + x**8*y**5 + x**9 + x**7*y + x**8*y**3 + x**9*y**4,     x**6*y**5 + x**7*y**5 + x**6 + x**6*y + x**7*y + x**6*y**4)
\bar Z_1 = Z(x**2*y**5 + x**6*y**5 + x**11*y + x**7*y + x**3 + x**7,     x**7*y**5 + x**3*y**5 + x**2*y**4 + x**6*y**4 + y + x**4*y)
\bar X_2 = X(x**3*y**5 + x**4*y**5 + x**5 + x**3*y + x**4*y**3 + x**5*y**4,     x**2*y**5 + x**3*y**5 + x**2 + x**2*y + x**3*y + x**2*y**4)
\bar Z_2 = Z(x*y**3 + x**5*y**3 + x**10*y**5 + x**6*y**5 + x**2*y**4 + x**6*y**4,     x**6*y**3 + x**2*y**3 + x*y**2 + x**5*y**2 + x**1

In [ ]:
def get_automorphisms_for_two_gross_code():
    """Logic unit for the 288-qubit code designed by Ted Yoder in January 2024."""
    assert current_code == "two-gross"

    if not toric:
        Lx = x+x**5*y+(x+x**7+x**8)*y**2+x**6*y**4+x**6*y**5+x**7*y**6+(x**2+x**3)*y**9
        Rx = x**4*y+(x**5+x**7+x**11)*y**2+x*y**3+x**8*y**4+x**6*y**6+x**3*y**7+x*y**8+x**8*y**9
        Lz = ((x**5+x**11)*y+(x**2+x**11)*y**3+(x**8+x**11)*y**6+(x**2+x**11)*y**8+(x**2+x**11)*y**11).T
        Rz = ((x**3+x**10)*y**2+(x+x**9)*y**5+(1+x**7)*y**8+(1+x**4+x**5+x**11)*y**11).T
    else:
        p = x**2+(x**2+x**8+x**9)*y**2+(x**3+x**4)*y**3+x**7*y**4+x**8*y**6+x**6*y**7+x**7*y**11
        q = x**3*y**2+(x**5+x**7+x**11)*y**3+x**8*y**4+x**8*y**5+x**6*y**7+x**4*y**8+x*y**9+x*y**10
        r = (x**4+x**11)*y**2+(1+x+x**5+x**6)*y**5+(x+x**8)*y**8+(x**2+x**10)*y**11
        s = (x**2+x**11)*(y**6+y**9+y**10)+(x**8+x**11)*y**7+(x**5+x**11)*y**8

        nu = x*y
        Lx = p
        Rx = q
        Lz = nu*s.T
        Rz = nu*r.T


    xshifts = [x**0, y**3, x**3*y**7, x**11*y**11, x**2*y**9, x**7*y**4]
    zshifts = [x**0, x*y, x**4, x**5*y**4, x**4*y**3, x**3*y**5]


    # Xs, Zs = get_block_basis_with_pivot(Lx, Rx, Lz, Rz)
    Xs, Zs = get_block_basis_with_shifts(Lx, Rx, Lz, Rz, xshifts, zshifts)

    print("x")
    print(get_linear_map_for_automorphism(x,Xs,Zs))

    print("\ny")
    print(get_linear_map_for_automorphism(y,Xs,Zs))

    print("\nBasis:")
    for i,((lx,rx),(lz,rz)) in enumerate(zip(Xs,Zs)):
        print(f"\\bar X_{i} = X({lx},     {rx})")
        print(f"\\bar Z_{i} = Z({lz},     {rz})")

get_automorphisms_for_two_gross_code()

x
[[0 1 1 1 0 1]
 [1 0 1 0 1 1]
 [1 0 1 0 1 0]
 [1 0 1 1 1 1]
 [0 1 1 1 1 1]
 [1 0 0 1 1 0]]

y
[[1 1 1 1 1 0]
 [1 1 0 1 1 1]
 [0 1 1 0 0 0]
 [1 0 0 0 1 0]
 [1 0 0 1 1 1]
 [1 0 0 0 0 1]]

Basis:
\bar X_0 = X(x**2 + x**2*y**2 + x**8*y**2 + x**9*y**2 + x**3*y**3 + x**4*y**3 + x**7*y**4 + x**8*y**6 + x**6*y**7 + x**7*y**11,     x**3*y**2 + x**5*y**3 + x**7*y**3 + x**11*y**3 + x**8*y**4 + x**8*y**5 + x**6*y**7 + x**4*y**8 + x*y**9 + x*y**10)
\bar Z_0 = Z(x**11*y**7 + x**11*y**4 + x**11*y**3 + x**2*y**7 + x**2*y**4 + x**2*y**3 + x**5*y**6 + x**2*y**6 + x**8*y**5 + x**2*y**5,     x**9*y**11 + x**2*y**11 + y**8 + x*y**8 + x**8*y**8 + x**7*y**8 + y**5 + x**5*y**5 + x**11*y**2 + x**3*y**2)
\bar X_1 = X(x**2*y**3 + x**2*y**5 + x**8*y**5 + x**9*y**5 + x**3*y**6 + x**4*y**6 + x**7*y**7 + x**8*y**9 + x**6*y**10 + x**7*y**2,     x**3*y**5 + x**5*y**6 + x**7*y**6 + x**11*y**6 + x**8*y**7 + x**8*y**8 + x**6*y**10 + x**4*y**11 + x + x*y)
\bar Z_1 = Z(y**8 + y**5 + y**4 + x**3*y**8 + x**3*y**5 + x**3*y*